<a href="https://colab.research.google.com/github/gsarm78/MetaSegmentation/blob/main/ProtoNet_SatImage_Experiment_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Data are renamed with a numbered filename
!unzip /content/drive/My\ Drive/cs330Data/data.zip -d /content/data

In [4]:
rm -r /content/data/__MACOSX

rm: cannot remove '/content/data/__MACOSX': No such file or directory


In [ ]:
!unzip /content/drive/My\ Drive/cs330Data/data2.zip -d /content/data2

In [22]:
rm -r /content/data2/__MACOSX

Dataloader

In [5]:
!git clone https://github.com/gsarm78/MetaSegmentation.git

Cloning into 'MetaSegmentation'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 40 (delta 15), reused 6 (delta 1), pack-reused 0
Unpacking objects: 100% (40/40), done.


In [7]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import tqdm
import matplotlib.pyplot as plt
from PIL import Image

In [8]:
pano_directory = '/content/data/data/train'
labels_directory = '/content/data/data/labels'
dataset_csv = '/content/MetaSegmentation/andrew_fewshot_dataloader/dataset_10_31_2020_3903_clustered.csv'
df = pd.read_csv('/content/MetaSegmentation/andrew_fewshot_dataloader/dataset_10_31_2020_3903_clustered.csv')

In [9]:
def get_image(path,IMG_WIDTH=255,IMG_HEIGHT=255,IMG_CHANNELS=3):
    '''
    '''
    img = Image.open(path)
    img = img.resize((IMG_WIDTH,IMG_HEIGHT))
    return img

def get_mask(path,IMG_WIDTH=255,IMG_HEIGHT=255):
    '''
    '''
    img = Image.open(path)
    img = img.resize((IMG_WIDTH,IMG_HEIGHT))
    img = np.array(img)/255.
    img = np.expand_dims(img[:,:,0],axis=-1)

    return img
def get_images(dataset,n_samples=16,shuffle=True):
    '''
    '''
    set = random.sample(dataset,n_samples)
    return set

In [43]:
class DataGenerator(object):
    def __init__(self,
                 dataset_csv,
                 pano_directory,
                 label_directory,
                 num_classes,
                 num_samples_per_class,
                 num_meta_test_classes,
                 num_meta_test_samples_per_class,
                 IMG_WIDTH=255,
                 IMG_HEIGHT=255,
                 num_circles=3):
        '''
        '''
        self.num_samples_per_class = num_samples_per_class
        self.num_classes = num_classes
        self.num_meta_test_samples_per_class = num_meta_test_samples_per_class
        self.num_meta_test_classes = num_meta_test_classes
        self.dataset_csv = dataset_csv
        self.num_circles=num_circles
        self.dataset_df = pd.read_csv(self.dataset_csv)
        # limit the dataset to only contain examples with N num_circles
        self.dataset_df = df[df['num_circles']==self.num_circles-1]# zero index
        self.pano_directory = pano_directory
        self.label_directory = label_directory
        
        self.IMG_WIDTH=IMG_WIDTH
        self.IMG_HEIGHT=IMG_HEIGHT
        data_ids = [os.path.join(pano_directory,i) for i in df['pano_name'].tolist()]
        label_ids =  [os.path.join(labels_directory,i) for i in df['label_name'].tolist()]
        self.dataset = list(zip(data_ids,label_ids))

        self.NUM = len(self.dataset)
        self.train_split = 0.8
        self.val_split = 0.1
        self.test_split = 0.1

        self.num_train = int(self.train_split*self.NUM)
        self.num_val = int(self.val_split*self.NUM)
        self.num_test = int(self.test_split*self.NUM)
        # print(NUM,num_train,num_val,num_test)
        random.seed(123)
        random.shuffle(self.dataset)
        self.train_dataset = self.dataset[:self.num_train]
        self.val_dataset = self.dataset[self.num_train:self.num_train+self.num_val]
        self.test_dataset = self.dataset[self.num_train+self.num_val+1:]

    def sample_batch(self,batch_type,batch_size,shuffle=True,swap=False):
        if batch_type =='meta_train':
            folders = self.train_dataset
            num_classes = self.num_classes
            num_samples_per_class = self.num_samples_per_class
        elif batch_type == 'meta_val':
            folders = self.val_dataset
            num_classes = self.num_classes
            num_samples_per_class = self.num_samples_per_class
        else:
            folders = self.test_dataset
            num_classes = self.num_meta_test_classes
            num_samples_per_class = self.num_meta_test_samples_per_class

        all_image_batches = []
        all_label_batches = []
        for i in range(batch_size):
            # dont need to random sample folders, since only one class
            labels_and_images = get_images(folders,n_samples=num_samples_per_class)
            labels = [get_mask(li[1]) for li in labels_and_images]
            images = [get_image(li[0]) for li in labels_and_images]

            labels = np.stack(labels).astype(np.int32)
            labels = np.reshape(labels,(num_classes,
                                        num_samples_per_class,
                                        self.IMG_WIDTH,
                                        self.IMG_HEIGHT,
                                        -1))
            images = np.stack(images)
            images = np.reshape(images,(num_classes,
                                        num_samples_per_class,
                                        self.IMG_WIDTH,
                                        self.IMG_HEIGHT,
                                        -1))
            print(labels.shape,images.shape)
            all_image_batches.append(images)
            all_label_batches.append(labels)
        all_image_batches = np.stack(all_image_batches)
        all_label_batches = np.stack(all_label_batches)
        print(all_image_batches.shape,all_label_batches.shape)
        return all_image_batches,all_label_batches

In [44]:
d = DataGenerator(dataset_csv=dataset_csv,
                 pano_directory=pano_directory,
                 label_directory=labels_directory,
                 num_classes=1,
                 num_samples_per_class=2*2,#2-shot
                 num_meta_test_classes=1,
                 num_meta_test_samples_per_class=3*2,#3-shot test
                 IMG_WIDTH=255,
                 IMG_HEIGHT=255,
                 num_circles=3)
batch_size = 4
batch_type = 'meta_val'
images, labels = d.sample_batch(batch_type,batch_size)
print(images.shape,labels.shape)

(1, 4, 255, 255, 1) (1, 4, 255, 255, 4)
(1, 4, 255, 255, 1) (1, 4, 255, 255, 4)
(1, 4, 255, 255, 1) (1, 4, 255, 255, 4)
(1, 4, 255, 255, 1) (1, 4, 255, 255, 4)
(4, 1, 4, 255, 255, 4) (4, 1, 4, 255, 255, 1)
(4, 1, 4, 255, 255, 4) (4, 1, 4, 255, 255, 1)


In [45]:
#ProtoNet
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers


class ProtoNet(tf.keras.Model):

  def __init__(self, num_filters, latent_dim):
    super(ProtoNet, self).__init__()
    self.num_filters = num_filters
    self.latent_dim = latent_dim
    num_filter_list = self.num_filters + [latent_dim]
    self.convs = []
    for i, num_filter in enumerate(num_filter_list):
      block_parts = [
        layers.Conv2D(
          filters=num_filter,
          kernel_size=3,
          padding='SAME',
          activation='linear'),
      ]

      block_parts += [layers.BatchNormalization()]
      block_parts += [layers.Activation('relu')]
      block_parts += [layers.MaxPool2D()]
      block = tf.keras.Sequential(block_parts, name='conv_block_%d' % i)
      self.__setattr__("conv%d" % i, block)
      self.convs.append(block)
    self.flatten = tf.keras.layers.Flatten()

  def call(self, inp):
    out = inp
    for conv in self.convs:
      out = conv(out)
    out = self.flatten(out)
    return out

def ProtoLoss(x_latent, q_latent, labels_onehot, num_classes, num_support, num_queries):
  """
    calculates the prototype network loss using the latent representation of x
    and the latent representation of the query set
    Args:
      x_latent: latent representation of supports with shape [N*S, D], where D is the latent dimension
      q_latent: latent representation of queries with shape [N*Q, D], where D is the latent dimension
      labels_onehot: one-hot encodings of the labels of the queries with shape [N, Q, N]
      num_classes: number of classes (N) for classification
      num_support: number of examples (S) in the support set
      num_queries: number of examples (Q) in the query set
    Returns:
      ce_loss: the cross entropy loss between the predicted labels and true labels
      acc: the accuracy of classification on the queries
  """
  q_labels = labels_onehot[:, :, -num_queries:]
  q_labels = tf.reshape(q_labels, (num_classes*num_queries, -1))
  q_latent = tf.reshape(q_latent, (num_classes*num_queries, 1, -1))
  q_latent = tf.tile(q_latent, (1, num_classes, 1))
  x_latent = tf.reshape(x_latent, (num_classes, num_support, -1))

  ck = tf.reduce_mean(x_latent, axis=1)
  centroids = tf.reshape(ck, (1, num_classes, -1))
  centroids = tf.tile(centroids, (num_classes*num_queries,1,1))
  # compute the distance from the prototypes
  distances = -tf.norm(q_latent - centroids, axis=-1)  #default to Euclidean distance (ord='euclidean')
  # compute cross entropy loss
  computed_ce_loss = tf.nn.softmax_cross_entropy_with_logits(labels=q_labels, logits=distances)
  #return the cross-entropy loss 
  ce_loss = tf.reduce_mean(computed_ce_loss)
  #return accuracy
  acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(q_labels, axis=-1), tf.argmax(distances, axis=-1)), dtype=tf.float32))
  return ce_loss, acc

In [52]:
# run_ProtoNet
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import os
import glob
import matplotlib.pyplot as plt

def proto_net_train_step(model, optim, x, q, labels_ph):
  num_classes, num_support, im_height, im_width, channels = x.shape
  num_queries = q.shape[1]
  x = tf.reshape(x, [-1, im_height, im_width, channels])
  q = tf.reshape(q, [-1, im_height, im_width, channels])

  with tf.GradientTape() as tape:
    x_latent = model(x)
    q_latent = model(q)
    ce_loss, acc = ProtoLoss(x_latent, q_latent, labels_ph, num_classes, num_support, num_queries)

  gradients = tape.gradient(ce_loss, model.trainable_variables)
  optim.apply_gradients(zip(gradients, model.trainable_variables))
  return ce_loss, acc

def proto_net_eval(model, x, q, labels_ph):
  num_classes, num_support, im_height, im_width, channels = x.shape
  num_queries = q.shape[1]
  x = tf.reshape(x, [-1, im_height, im_width, channels])
  q = tf.reshape(q, [-1, im_height, im_width, channels])

  x_latent = model(x)
  q_latent = model(q)
  ce_loss, acc = ProtoLoss(x_latent, q_latent, labels_ph, num_classes, num_support, num_queries)

  return ce_loss, acc 

def run_protonet(n_way=1, k_shot=2, n_query=2, n_meta_test_way=1, k_meta_test_shot=3, n_meta_test_query=3):
  n_epochs = 20
  n_episodes = 100

  im_width, im_height, channels = 255, 255, 3
  num_filters = 32
  latent_dim = 16
  num_conv_layers = 3
  n_meta_test_episodes = 1000

  model = ProtoNet([num_filters]*num_conv_layers, latent_dim)
  optimizer = tf.keras.optimizers.Adam()

  writer = tf.summary.create_file_writer(f'/content/logs/')

  with writer.as_default():
    # call DataGenerator with k_shot+n_query samples per class
    data_generator = DataGenerator(dataset_csv=dataset_csv,
                                   pano_directory=pano_directory,
                                   label_directory=labels_directory,
                                   num_classes=n_way, 
                                   num_samples_per_class=k_shot+n_query, 
                                   num_meta_test_classes=n_meta_test_way, 
                                   num_meta_test_samples_per_class=k_meta_test_shot+n_meta_test_query,
                                   IMG_WIDTH=255, IMG_HEIGHT=255, num_circles=3)
    for ep in range(n_epochs):
      for epi in range(n_episodes):
        # sample a batch of training data and partition it into
        # support and query sets

        images, labels = data_generator.sample_batch(batch_type='meta_train', batch_size=4, shuffle=False, swap=False) 
        
        #images = images.reshape(n_way, k_shot+n_query, im_width, im_height, channels)

        support = images[:,:k_shot]
        query = images[:,-n_query:]

        ls, ac = proto_net_train_step(model, optimizer, x=support, q=query, labels_ph=labels)
        if (epi+1) % 50 == 0:
          #############################
          # sample a batch of validation data and partition it into
          # support and query sets
          images, labels = data_generator.sample_batch(batch_type='meta_val', batch_size=4, shuffle=False, swap=False)
          images = images.reshape(n_way, k_shot + n_query, im_width, im_height, channels)
          support = images[:,:k_shot]
          query = images[:,-n_query:]
          #############################
          val_ls, val_ac = proto_net_eval(model, x=support, q=query, labels_ph=labels)
          print('[epoch {}/{}, episode {}/{}] => meta-training loss: {:.5f}, meta-training acc: {:.5f}, meta-val loss: {:.5f}, meta-val acc: {:.5f}'.format(ep+1,
                                                                      n_epochs,
                                                                      epi+1,
                                                                      n_episodes,
                                                                      ls,
                                                                      ac,
                                                                      val_ls,
                                                                      val_ac))
          ## Adding Logging
          tf.summary.scalar('Validation Loss', val_ls, step=ep*n_episodes+epi)
          tf.summary.scalar('Validation Accuracy', val_ac, step=ep*n_episodes+epi)
          writer.flush()

  print('Testing...')
  meta_test_accuracies = []
  for epi in range(n_meta_test_episodes):

    # sample a batch of test data and partition it into
    # support and query sets
    images, labels = data_generator.sample_batch(batch_type='meta_test', batch_size=1, shuffle=False,swap=False)
    images = images.reshape(n_way, k_meta_test_shot + n_meta_test_query, im_width, im_height, channels)
    support = images[:,:k_meta_test_shot]
    query = images[:,-n_meta_test_query:]

    ls, ac = proto_net_eval(model, x=support, q=query, labels_ph=labels)
    meta_test_accuracies.append(ac)
    if (epi+1) % 50 == 0:
      print('[meta-test episode {}/{}] => loss: {:.5f}, acc: {:.5f}'.format(epi+1, n_meta_test_episodes, ls, ac))
  avg_acc = np.mean(meta_test_accuracies)
  stds = np.std(meta_test_accuracies)
  print('Average Meta-Test Accuracy: {:.5f}, Meta-Test Accuracy Std: {:.5f}'.format(avg_acc, stds))

In [53]:
run_protonet(n_way=1, k_shot=2, n_query=2, n_meta_test_way=1, k_meta_test_shot=3, n_meta_test_query=3)

(1, 4, 255, 255, 1) (1, 4, 255, 255, 4)
(1, 4, 255, 255, 1) (1, 4, 255, 255, 4)
(1, 4, 255, 255, 1) (1, 4, 255, 255, 4)
(1, 4, 255, 255, 1) (1, 4, 255, 255, 4)
(4, 1, 4, 255, 255, 4) (4, 1, 4, 255, 255, 1)


ValueError: ignored